In [1]:
from pathlib import Path
from jaal import Jaal
import pandas as pd
from bookgraph.reader import YaneuraBookReader
from bookgraph.util import get_project_root
from cshogi import Board

/workspaces/BookGraph/.venv/lib/python3.12/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
reader = YaneuraBookReader()

In [3]:
path = get_project_root() / Path("data/book.db")
nodes = list(reader.from_file(path))

from functools import reduce
from typing import Callable, Iterable
f: Callable[[list, list], list] = lambda li1, li2: li1 + li2
moves = reduce(f, (node.candidate_moves for node in nodes), []) #type: ignore

In [4]:
df = pd.DataFrame(nodes, columns=["node"])

In [5]:
df["board"] = df["node"].map(lambda x: x.sfen_for_hash)
df["next_move"] = df["node"].map(lambda x: x.candidate_moves[0].chosen_move_code)

In [6]:
df

,node,board,next_move
0,<bookgraph.book_tree.Node object at 0x70361c31...,+Bn1g1g1nl/2s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPP...,R*9b
1,<bookgraph.book_tree.Node object at 0x7035f280...,+Bn1g1g1nl/2s2k1s1/pp1pppbp1/6p1p/9/9/PP1PPPPP...,3c9i+
2,<bookgraph.book_tree.Node object at 0x7035f233...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b1p2/2P6/PP2PP...,3e3f
3,<bookgraph.book_tree.Node object at 0x7035f231...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b4/2P3p2/PP2PP...,3f3g+
4,<bookgraph.book_tree.Node object at 0x7035f234...,+Bn1g1g1nl/2sk2rs1/pppppp1pp/9/4b4/2P3p2/PP2PP...,3i2h
...,...,...,...
130177,<bookgraph.book_tree.Node object at 0x7035ea45...,lr4snl/3kg1gb1/p1nspp1pp/1ppp2p2/7P1/2PPP4/PPS...,9c9d
130178,<bookgraph.book_tree.Node object at 0x7035ea45...,rn1g1g1nl/3s1k1b1/p2p3pp/1pp1Rpp2/9/2P6/PP1P1P...,B*4e
130179,<bookgraph.book_tree.Node object at 0x7035ea45...,rn1g1g1nl/3s1k1b1/p2p3pp/1pp1spp2/9/2P6/PP1P1P...,5h5d
130180,<bookgraph.book_tree.Node object at 0x7035ea45...,rn1gkg1nl/2s3rs1/pppppp1pp/6p2/7P1/2P3P2/PP1PP...,9a9b


In [7]:
moves = []
for moves_list in (node.candidate_moves for node in nodes):
    moves += moves_list
moves = moves[:100]

In [8]:
print(moves[0])

sfen_for_hash: +Bn1g1g1nl/2s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL b RLPrlp
next_sfen_for_hash: +Bn1g1g1nl/R1s2k1s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL w LPrlp
chosen_move_code: R*9b
expected_next_move_code: 4b3a
evaluation_value: 0
metadata: {'0': '32'}



In [9]:
board_set = set(move.sfen_for_hash for move in moves) | set(move.next_sfen_for_hash for move in moves)
sfen_to_id_dict = {sfen_for_hash: idx for idx, sfen_for_hash in enumerate(board_set)}

In [10]:
sfen_to_id_dict

{'1n1g1gknl/R+Bs4s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL w LPrlp': 0,
 '1ns2g1nl/lkg2srb1/pppppp1pp/9/1P4p2/2PP5/P1BSPPPPP/1R3K3/LN1G1GSNL b -': 1,
 '1n1g1g1nl/1Bskr2s1/lppp1p1pp/p5p2/7P1/P1P6/1PSP1PP1P/5S1R1/LNKG1G1NL b Pbp': 2,
 '+Bn1g1g1nl/2sk3s1/ppppp2pp/9/6p2/2P2r1P1/PP1PPpP1P/1K5R1/LNSGSG1NL w LPb': 3,
 '1ns2gsnl/l1kg1r1b1/pp1pp2pp/2p2pp2/9/4P2P1/PP1PSPP1P/1BR6/LNSGKG1NL b p': 4,
 'kns2g1nl/l1g2s1b1/pppppp1pp/6r2/1P4p2/2PP5/P1BSPPPPP/1R2GKS2/LN3G1NL b -': 5,
 '1ns2g1nl/lkg2srb1/pppppp1pp/6p2/9/2PP1P3/PPBSP1PPP/1R2G1S2/LN2KG1NL w -': 6,
 '1npg1gknl/R+Bs4s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP/9/+bNSGKGSNL b LPrl': 7,
 'kns2g1nl/l1g2srb1/pppppp1pp/9/1P4p2/2PP5/P1BSPPPPP/1R3K1S1/LN1G1G1NL b -': 8,
 '1ns4nl/lkg1gsrb1/pppppp1pp/9/1P4p2/2PP5/P1BSPPPPP/1R3KS2/LN1G1G1NL b -': 9,
 '1ns2g1nl/lk1g1rsb1/ppppp2pp/5pp2/9/4P1PP1/PP1PSP2P/1BR6/LNSGKG1NL b P': 10,
 '1n1g1g1nl/1Bskr2s1/lppp1p1pp/p5p2/7P1/P1P1P4/1P1P1PP1P/2S2S1R1/LNKG1G1NL w Pb': 11,
 '1n1g1g1nl/1Bsk3s1/lppp1p1pp/p5p2/7P1/P1P1r4/1PSP1

In [11]:
df_move = pd.DataFrame(moves, columns=["move"])

In [12]:
df_move["from"] = df_move["move"].map(lambda x: sfen_to_id_dict[x.sfen_for_hash])
df_move["to"] = df_move["move"].map(lambda x: sfen_to_id_dict[x.next_sfen_for_hash])

In [13]:
edge_df = df_move[["from", "to"]].copy()

In [14]:
node_df = pd.DataFrame(sfen_to_id_dict.keys(), columns=["title"], index=sfen_to_id_dict.values())

In [15]:
node_df["id"] = node_df.index

In [16]:
node_df

,title,id
0,1n1g1gknl/R+Bs4s1/pp1ppp1p1/6p1p/9/9/PP1PPPPPP...,0
1,1ns2g1nl/lkg2srb1/pppppp1pp/9/1P4p2/2PP5/P1BSP...,1
2,1n1g1g1nl/1Bskr2s1/lppp1p1pp/p5p2/7P1/P1P6/1PS...,2
3,+Bn1g1g1nl/2sk3s1/ppppp2pp/9/6p2/2P2r1P1/PP1PP...,3
4,1ns2gsnl/l1kg1r1b1/pp1pp2pp/2p2pp2/9/4P2P1/PP1...,4
...,...,...
121,1ns2gsnl/l1kg1r1b1/ppppp2pp/2P2pp2/9/4P2P1/PP1...,121
122,kns2g1nl/l1g3rb1/ppppsp1pp/4p4/PP4p2/2PP5/2BSP...,122
123,1ns2g1nl/lkg3rb1/ppppsp1pp/4p4/PP4p2/2PP5/2BSP...,123
124,1n1g1g1nl/1Bskr2s1/lppp1p1pp/p5p2/4p2P1/P1P6/1...,124


In [17]:
node_df["board"] = node_df["title"].map(lambda x: Board(sfen=x))

In [18]:
node_df.iloc[0]["board"].to_svg().__str__()

'<svg xmlns="http://www.w3.org/2000/svg" version="1.1" xmlns:xlink="http://www.w3.org/1999/xlink" width="230.0" height="192.0" viewBox="0 0 230 192"><defs><g id="black-pawn"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#27497;</text></g><g id="black-lance"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#39321;</text></g><g id="black-knight"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#26690;</text></g><g id="black-silver"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#37504;</text></g><g id="black-gold"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#37329;</text></g><g id="black-bishop"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#35282;</text></g><g id="black-rook"><text font-family="serif" font-size="17" text-anchor="middle" x="10.5" y="16.5">&#39131;</text></g><g id="black

In [19]:
Jaal(edge_df, node_df).plot()

Parsing the data...Done


/workspaces/BookGraph/.venv/lib/python3.12/site-packages/jaal/datasets/parse_dataframe.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['73' '45' '74' '47' '86' '116' '53' '3' '101' '115' '51' '72' '106' '50'
 '12' '41' '37' '124' '11' '88' '60' '61' '0' '97' '38' '71' '77' '34'
 '107' '57' '27' '111' '7' '84' '67' '82' '16' '78' '21' '21' '21' '42'
 '80' '120' '33' '112' '79' '25' '49' '43' '43' '56' '32' '66' '10' '87'
 '104' '92' '29' '108' '69' '48' '75' '17' '114' '63' '62' '62' '26' '6'
 '99' '13' '76' '31' '1' '1' '1' '81' '81' '64' '18' '89' '110' '39' '54'
 '58' '28' '20' '23' '65' '68' '123' '125' '36' '90' '59' '4' '121' '105'
 '117']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  edge_df.loc[:, ['from', 'to']] = edge_df.loc[:, ['from', 'to']].astype(str)
/workspaces/BookGraph/.venv/lib/python3.12/site-packages/jaal/datasets/parse_dataframe.py:42: FutureWar

[2024-05-26 09:09:51,108] ERROR in app: Exception on /_dash-layout [GET]
Traceback (most recent call last):
  File "/workspaces/BookGraph/.venv/lib/python3.12/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/BookGraph/.venv/lib/python3.12/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/BookGraph/.venv/lib/python3.12/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/BookGraph/.venv/lib/python3.12/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/workspaces/BookGraph/.v